# Autotagging projects
* Who is the lead agency?
* Is this project on or off the SHS? Or even both?

In [1]:
import pandas as pd

# Settings
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = "{:,.2f}".format

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"
FILE = "fake_data.xlsx"

# My utilities
import _utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
# Read in file
df = pd.read_excel(f"{GCS_FILE_PATH}{FILE}", sheet_name="fake")

In [3]:
# Subset
df2 = df[['project_name', 'lead_agency','primary_mode',
       'secondary_mode_s_','shs_capacity_increase_detail',]]

In [4]:
def tagging_columns(df, tagging_col:str, new_col:str, keyword_list: list):
    
    # Delinate items in keywords list using |
    keywords = f"({'|'.join(keyword_list)})"
    
    # New column that captures whether or not the keyword appears
    df["keyword_appears_bool"] = df[tagging_col].str.contains(keywords)
    
    # Function to categorize whether something is on SHS or not
    def on_SHS_or_not(row):
        if row["keyword_appears_bool"] == False:
            return "highway related"
        else:
            return "not highway related"
             
    # Apply function 
    df[new_col] = df.apply(lambda x: on_SHS_or_not(x), axis=1)
             
    # Drop keyword col
    df = df.drop(columns = ["keyword_appears_bool"]) 
   
    return df 

In [5]:
df3 = tagging_columns(df2, "primary_mode", "primary_mode_SHS", ['Highway', 'Its',
       'Interchange',] ) 

/tmp/ipykernel_1181/889075973.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
/tmp/ipykernel_1181/889075973.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1181/889075973.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
df3 = tagging_columns(df2, "secondary_mode_s_", "secondary_mode_SHS", ["Highway"])

/tmp/ipykernel_1181/889075973.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
/tmp/ipykernel_1181/889075973.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1181/889075973.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
def caltrans_or_partner(row):
        if (row.lead_agency == "Caltrans"):
            return "Caltrans"
        if (row.lead_agency > ) and (row.shs_capacity_increase_detail != "None") and (row.caltrans_or_partner == "Caltrans"):
            return "on SHS"
        if (row.primary_mode_SHS == "highway related") and (row.shs_capacity_increase_detail != "None")  and (row.caltrans_or_partner == "Caltrans"):
            return "on SHS"
        if (row.secondary_mode_SHS == "highway related") and (row.primary_mode_SHS == "on_highway") and (row.shs_capacity_increase_detail != "None")  and (row.caltrans_or_partner == "Caltrans"):
            return "on SHS"
        else:
            return "not on SHS"
    

In [7]:
def SHS_sentence(df): 
    # Tag if it's Caltrans or a partner project
    df['caltrans_or_partner'] = df['lead_agency'].map(lambda x: 'Caltrans' if x == 'Caltrans' else 'a partner')     
    
    # If two or more columns are tagged as "on SHS",  label the project as on SHS. 
    def on_highway(row):
        if (row.secondary_mode_SHS== "highway related") and (row.primary_mode_SHS == "highway related") and (row.caltrans_or_partner == "Caltrans"):
            return "on SHS"
        if (row.secondary_mode_SHS == "highway related") and (row.shs_capacity_increase_detail != "None") and (row.caltrans_or_partner == "Caltrans"):
            return "on SHS"
        if (row.primary_mode_SHS == "highway related") and (row.shs_capacity_increase_detail != "None")  and (row.caltrans_or_partner == "Caltrans"):
            return "on SHS"
        if (row.secondary_mode_SHS == "highway related") and (row.primary_mode_SHS == "on_highway") and (row.shs_capacity_increase_detail != "None")  and (row.caltrans_or_partner == "Caltrans"):
            return "on SHS"
        else:
            return "not on SHS"
    
    # Apply the function
    df["On_SHS"] = df.apply(on_highway, axis=1)
    
    # Create a sentence that summarizes the lead agency and whether the project is on the SHS or not. 
    df['sentence'] = 'The lead agency is ' + df['caltrans_or_partner'] + ' and the project is ' + df['On_SHS'] + '.'
    
    return df

In [8]:
SHS_sentence(df3).sample(10)

,project_name,lead_agency,primary_mode,secondary_mode_s_,shs_capacity_increase_detail,primary_mode_SHS,secondary_mode_SHS,caltrans_or_partner,On_SHS,sentence
208,Ker 99 North Bakersfield Widening,None,Highway,None,General Purpose Lane,not highway related,highway related,a partner,not on SHS,The lead agency is a partner and the project is not on SHS.
544,Norwalk & Fullerton Over/Under,None,Grade Crossing,None,None,highway related,highway related,a partner,not on SHS,The lead agency is a partner and the project is not on SHS.
241,Lindsay And Route 198/245 Operational Improvements,None,Highway,None,None,not highway related,highway related,a partner,not on SHS,The lead agency is a partner and the project is not on SHS.
57,Yuba 70 Segments 4 And 5,None,Highway,None,General Purpose Lane,not highway related,highway related,a partner,not on SHS,The lead agency is a partner and the project is not on SHS.
145,Goleta Overcrossing,Lossan,Highway,None,Interchange (New),not highway related,highway related,a partner,not on SHS,The lead agency is a partner and the project is not on SHS.
719,Hiksari To Broadway Trail,None,Bike/Pedestrian,None,None,highway related,highway related,a partner,not on SHS,The lead agency is a partner and the project is not on SHS.
352,"Banning, Ca Raising Economic Opportunity: Hargrave Grade Separation Planning",None,Grade Separation,Rail (Freight),None,highway related,highway related,a partner,not on SHS,The lead agency is a partner and the project is not on SHS.
294,Sr-14 (Sr-138) Avenue M Interchange Improvements Project,None,Interchange (Widening),Bike/Pedestrian,Interchange (Widening),not highway related,highway related,a partner,not on SHS,The lead agency is a partner and the project is not on SHS.
340,Port Of Hueneme Intermodal Infrastructure Project: Wharf And Berth Improvements,Port Of Hueneme,Port,None,None,highway related,highway related,a partner,not on SHS,The lead agency is a partner and the project is not on SHS.
37,Bcag Zero Emission Electric Bus And Equipment (Butte County Association Of Governments),None,Transit,ZEV,None,highway related,highway related,a partner,not on SHS,The lead agency is a partner and the project is not on SHS.


In [9]:
df3[['lead_agency']].drop_duplicates()

,lead_agency
0,Caltrans
1,Tolowa Dee-Ni Nation
7,Humboldt Transit Authority
11,Caltrans\nCity Of Eureka
16,Caltrans\nLake County
22,Mendocino Council Of Governments
35,Trinity County
37,None
39,Nevada County Transportation Commission
40,Placer County
